RNN의 입력단위를 단어 단위로 사용해보자.

### 1. 훈련 데이터 전처리하기

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

실습을 위해 임의의 문장을 만들자.

In [2]:
sentence = 'Repeat is the best medicine for memory'.split()

우리가 만들 RNN은 'Repeat is the besst medicine for'을 입력받으면  
'is the best medicine for memory'를 출력하는 RNN이다.  
위의 임의의 문장으로부터 vocabulary를 만들자.  

In [3]:
vocab = list(set(sentence))
print(vocab)

['Repeat', 'memory', 'medicine', 'is', 'best', 'the', 'for']


이제 단어장의 단어에 고유한 정수 인덱스를 부여하자.  
UNK 토큰도 추가한다.  

In [4]:
word2index = {tkn: i for i, tkn in enumerate(vocab,1)} # 단어에 고유한 정수 부여
word2index['<unk>'] = 0

In [5]:
print(word2index)

{'Repeat': 1, 'memory': 2, 'medicine': 3, 'is': 4, 'best': 5, 'the': 6, 'for': 7, '<unk>': 0}


word2index가 우리가 사용할 최종 단어장인 셈이다.  
word2index에 단어를 입력하면 맵핑되는 정수가 return된다.  

In [6]:
print(word2index['memory'])

2


예측 단계에서 예측한 문장을 확인하기 위해 idx2word도 만든다.  

In [7]:
index2word = {v:k for k, v in word2index.items()}
print(index2word)

{1: 'Repeat', 2: 'memory', 3: 'medicine', 4: 'is', 5: 'best', 6: 'the', 7: 'for', 0: '<unk>'}


idx2word는 정수로부터 단어를 리턴하는 역할을 한다.

In [8]:
print(index2word[2])

memory


정수 2와 매핑되는 단어가 memory인것을 확인했다.  
이제 데이터의 각 단어를 정수로 인코딩하는 동시에,  
입력 데이터와 레이블 데이터를 만드는 build_data라는 함수를 만들자.  

In [9]:
def build_data(sentence, word2index):
    encoded = [word2index[token] for token in sentence] # 각 문자를 정수로 변환.  
    input_seq, label_seq = encoded[:-1], encoded[1:] # 입력 시퀀스와 레이블 시퀀스를 분리
    input_seq = torch.LongTensor(input_seq).unsqueeze(0) # 배치 차원 추가, unsqueeze는 차원 추가시 쓰는 메소드
    label_seq = torch.LongTensor(label_seq).unsqueeze(0) # 배치 차원 추가

    return input_seq, label_seq

만들어진 함수로부터 입력 데이터와 레이블 데이터를 얻는다. 

In [10]:
X, Y = build_data(sentence, word2index)

입력 데이터와 레이블 데이터가 정상적으로 생성되었는지 출력해보자.  

In [ ]:
print(X) # Repeat is the best medicine for을 의미
print(Y) # is the best medicine for memory을 의미

tensor([[1, 4, 6, 5, 3, 7]])
tensor([[4, 6, 5, 3, 7, 2]])


### 2. 모델 구현하기

이전 모델들과 달라진 점은 임베딩 층을 추가했다는 점이다.  
파이토치에서는 nn.Embedding()을 사용해서 임베딩 층을 구현한다.  
nn.Embedding()은 크게 2개의 인자를 받는데   
첫 번째 인자는 vocab 크기이며, 두 번째 인자는 임베딩 벡터의 차원이다.  

In [ ]:
class Net(nn.Module):
    def __init__(self, vocab_size, input_size, hidden_size, batch_first=True):
        super(Net, self).__init__()
        self.embedding_layer = nn.Embedding(
            num_embeddings=vocab_size, # 워드 임베딩
            embedding_dim=input_size
            )
        self.rnn_layer = nn.RNN(input_size, hidden_size, batch_first=batch_first) # 입력 차원, 은닉 상태의 크기 정의

        self.linear = nn.Linear(hidden_size, vocab_size) # 출력은 원-핫 벡터의 크기를 가져야함. 또는 단어 집합의 크기만큼 가져야함.  

    def forward(self, x):
        # 1. 임베딩 층
        # 크기 변화 : (배치 크기, 시퀀스 길이) -> (배치 크기, 시퀀스 길이, 임베딩 차원)
        output = self.embedding_layer(x)
        # 2. RNN 층
        # 크기 변화 : (배치 크기, 시퀀스 길이, 임베딩 차원) -> output (배치 크기, 시퀀스 길이, 은닉층 크기), hidden (1, 배치 크기, 은닉층 크기)
        output, hidden = self.rnn_layer(output)
        # 3. 최종 출력층
        # 크기 변화 : (배치 크기, 시퀀스 길이, 은닉층 크기) -> (배치 크기, 시퀀스 길이, vocab 크기)
        output = self.linear(output)
        # 4. view를 통해서 배치 차원 제거
        # 크기 변화 : (배치 크기, 시퀀스 길이, 단어장 크기) -> (배치 크기 * 시퀀스 길이, vocab 크기)
        return output.view(-1, output.size(2))   

모델을 위한 하이퍼파라미터를 설정한다. 

In [34]:
# 하이퍼파라미터
vocab_size = len(word2index) # 단어장의 크기는 임베딩 층, 최종 출력층에 사용된다. <unk> 토큰을 크기에 포함한다.  
input_size = 5 # 임베딩 된 차원의 크기 및 RNN 층 입력 차원의 크기
hidden_size = 20 # RNN의 은닉층 크기

모델을 생성하자.

In [37]:
# 모델 생성
model = Net(vocab_size, input_size, hidden_size, batch_first=True)
# 손실 함수 정의 
loss_function = nn.CrossEntropyLoss() # 소프트맥스 함수 포함이며 실제값은 원-핫 인코딩 안해도 됨 

# 옵티마이저 정의
optimizer = optim.Adam(params=model.parameters())

모델에 입력을 넣어서 출력을 확인해보자.

In [38]:
# 임의로 예측해보기. 가중치는 전부 랜덤 초기화 된 상태이다.  
output = model(X)
print(output)

tensor([[-0.1192, -0.0755,  0.1736,  0.0682, -0.1780, -0.1769,  0.0482, -0.0649],
        [ 0.0842, -0.1051,  0.2078,  0.0186, -0.0048, -0.1662,  0.2576, -0.2520],
        [-0.1753, -0.1870,  0.3028,  0.0995, -0.4301, -0.0730,  0.2211, -0.4187],
        [-0.3211, -0.0823,  0.3491,  0.3673, -0.2930, -0.0967,  0.0703, -0.1347],
        [-0.1513,  0.1298,  0.3121,  0.2994, -0.1947, -0.0075,  0.3203,  0.0009],
        [ 0.0867, -0.0684,  0.0039,  0.0147,  0.1446, -0.1957,  0.0519, -0.1497]],
       grad_fn=<ViewBackward0>)


현재 weight는 랜덤 초기화되어 있어 의미는 없다.  
예측값의 크기를 확인해보자.  

In [39]:
print(output.shape)

torch.Size([6, 8])


이는 각각 (시퀀스의 길이, 은닉층의 크기)에 해당한다.  
모델은 훈련시키기 전에 예측을 제대로 하고 있는지 예측된 정수 시퀀스를 다시 단어 시퀀스로 바꾸는 decode 함수를 만든다.  

In [40]:
# 수치화된 데이터를 단어로 전환하는 함수
decode = lambda y : [index2word.get(x) for x in y]

약 200 에포크 학습하자.

In [41]:
# 훈련 시작
for step in range(201):
    # 경사 초기화
    optimizer.zero_grad()
    # 순방향 전파
    output = model(X)
    # 손실값 계산
    loss = loss_function(output, Y.view(-1))
    # 역방향 전파
    loss.backward()
    # 매개변수 업데이트 
    optimizer.step()
    # 기록
    if step % 40 == 0:
        print("[{:02d}/201] {:.4f} ".format(step+1, loss))
        pred = output.softmax(-1).argmax(-1).tolist()
        print(" ".join(["Repeat"] + decode(pred)))
        print()



[01/201] 2.0244 
Repeat memory the memory medicine the is

[41/201] 1.4555 
Repeat memory the best medicine for memory

[81/201] 0.8057 
Repeat is the best medicine for memory

[121/201] 0.3762 
Repeat is the best medicine for memory

[161/201] 0.1899 
Repeat is the best medicine for memory

[201/201] 0.1116 
Repeat is the best medicine for memory

